```
22/02/03
1. 베이스라인 제출(랜덤 포레스트) = 0.639
2. 정규화 후 제출(랜덤 포레스트) = 0.62
3. 이상치 제거 후 제출(랜덤 포레스트) = 0.5933
4. 이상치 제거 + 정규화 후 제출해보기 = 0.535 
5. 정규화 + lightgbm = 0.644 
6. 정규화 + 보팅(xgb,lightgmb,rf) = 0.662

22/02/04
1. 보팅(xgb,lightgmb,rf) = 0.699 (best - 24th)
2. gridsearch + rf = 0.627
3. 하이퍼파라미터 + 보팅 = 0.686
4, 보팅(모델 추가) = 0.676
```

```
# 변수 설명

index 구분자
quality 품질
fixed acidity 산도
volatile acidity 휘발성산
citric acid 시트르산
residual sugar 잔당 : 발효 후 와인 속에 남아있는 당분
chlorides 염화물
free sulfur dioxide 독립 이산화황
total sulfur dioxide 총 이산화황
density 밀도
pH 수소이온농도
sulphates 황산염
alcohol 도수
type 종류
```

In [ ]:
import pandas as pd

import matplotlib as mpl
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
print(mpl.__version__)
# seaborn 스타일 사용
plt.style.use('seaborn-notebook')

# matplotlib 그래프 한글폰트 깨질 때 대처(Mac & Window)
import platform
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
# Mac 인 경우
    rc('font', family='AppleGothic')
    
# 그래프에서 마이너스 기호가 표시되도록 하는 설정
mpl.rcParams['axes.unicode_minus'] = False

import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline


# 베이스라인 코드

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
train.head()

In [ ]:
plt.figure(figsize=(12,12))
sns.heatmap(data = train.corr(), annot=True)

In [ ]:
plt.figure(figsize=(12,12))
sns.boxplot(data = train[''])

In [ ]:
plt.figure(figsize=(12,12))
for i in range(1,13):
    plt.subplot(3,4,i)
    sns.distplot(train.iloc[:,i])
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
enc.fit(train['type'])
train['type'] = enc.transform(train['type'])
test['type'] = enc.transform(test['type'])

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.isna().sum()

In [ ]:
test.isna().sum()

In [ ]:
X_train = train.drop(['index','quality'], axis = 1)
X_test = test.drop(['index'], axis = 1)
y_train = train['quality']

In [ ]:
X_train

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dt_clf = DecisionTreeClassifier(random_state=0)
dt_clf.fit(X_train, y_train)

In [ ]:
pred = dt_clf.predict(X_test)

In [ ]:
pred

In [ ]:
dt_clf.feature_importances_

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf_clf = RandomForestClassifier(random_state=0, max_depth=10) 
rf_clf.fit(X_train , y_train)
pred = rf_clf.predict(X_test)
pred

In [ ]:
submission = pd.read_csv('sample_submission.csv')

In [ ]:
submission['quality'] = pred

In [ ]:
submission

In [ ]:
submission.to_csv('submission.csv',index=False)

## 전처리 시작 

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
enc.fit(train['type'])
train['type'] = enc.transform(train['type'])
test['type'] = enc.transform(test['type'])

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(train[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']])  # 데이터프레임이 되어버림

train[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']] = scaler.transform(train[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']])


train

In [ ]:
test

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(test[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']])  # 데이터프레임이 되어버림

test[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']] = scaler.transform(test[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']])


test

In [ ]:
train.head()

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
X_train = train.drop(['index','quality'], axis = 1)
X_test = test.drop(['index'], axis = 1)

In [ ]:
y_train = train['quality']

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(random_state=0, max_depth=10) 
rf_clf.fit(X_train , y_train)
pred = rf_clf.predict(X_test)
pred

In [ ]:
submission = pd.read_csv('sample_submission.csv')

submission['quality'] = pred

submission

submission.to_csv('submission.csv',index=False)

## X_train 이상치 제거, y_train index 맞춤

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
enc.fit(train['type'])
train['type'] = enc.transform(train['type'])
test['type'] = enc.transform(test['type'])

In [ ]:
X_train = train.drop(['index','quality'], axis = 1)
X_test = test.drop(['index'], axis = 1)
y_train = train['quality']

In [ ]:
for i in range(len(X_train.columns)-1):
    quantile_25 = np.quantile(X_train[X_train.columns[i]] , 0.25)
    quantile_75 = np.quantile(X_train[X_train.columns[i]] , 0.75)
    IQR = quantile_75 - quantile_25
    minimum = quantile_25 - 1.5 * IQR
    maximum = quantile_75 + 1.5 * IQR
    X_train_outer = X_train[(X_train[X_train.columns[i]] < minimum) | (X_train[X_train.columns[i]] > maximum)]
    X_train = X_train.drop(X_train_outer.index, axis = 0)

In [ ]:
X_train

In [ ]:
y_train = y_train[X_train.index]

In [ ]:
y_train

In [ ]:
X_test

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(random_state=0, max_depth=10) 
rf_clf.fit(X_train , y_train)
pred = rf_clf.predict(X_test)
pred

In [ ]:
submission = pd.read_csv('sample_submission.csv')

submission['quality'] = pred

submission

submission.to_csv('submission.csv',index=False)

# X_train 이상치제거 + 정규화

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
enc.fit(train['type'])
train['type'] = enc.transform(train['type'])
test['type'] = enc.transform(test['type'])

In [ ]:
X_train = train.drop(['index','quality'], axis = 1)
X_test = test.drop(['index'], axis = 1)
y_train = train['quality']

In [ ]:
for i in range(len(X_train.columns)-1):
    quantile_25 = np.quantile(X_train[X_train.columns[i]] , 0.25)
    quantile_75 = np.quantile(X_train[X_train.columns[i]] , 0.75)
    IQR = quantile_75 - quantile_25
    minimum = quantile_25 - 1.5 * IQR
    maximum = quantile_75 + 1.5 * IQR
    X_train_outer = X_train[(X_train[X_train.columns[i]] < minimum) | (X_train[X_train.columns[i]] > maximum)]
    X_train = X_train.drop(X_train_outer.index, axis = 0)

In [ ]:
X_train

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(X_train[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']])

X_train[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']] = scaler.transform(X_train[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']])


X_train

In [ ]:
y_train = y_train[X_train.index]

In [ ]:
y_train

In [ ]:
X_test

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(X_test[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']])  # 데이터프레임이 되어버림

X_test[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']] = scaler.transform(X_test[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']])


X_test

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(random_state=0, max_depth=5) 
rf_clf.fit(X_train , y_train)
pred = rf_clf.predict(X_test)
pred

In [ ]:
submission = pd.read_csv('sample_submission.csv')

submission['quality'] = pred

submission

submission.to_csv('submission.csv',index=False)

# import lightgbm as lgbm

In [ ]:
import lightgbm as lgbm

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
enc.fit(train['type'])
train['type'] = enc.transform(train['type'])
test['type'] = enc.transform(test['type'])

In [ ]:
X_train = train.drop(['index','quality'], axis = 1)
X_test = test.drop(['index'], axis = 1)
y_train = train['quality']

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(X_train[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']])

X_train[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']] = scaler.transform(X_train[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']])


X_train

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(X_test[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']])  # 데이터프레임이 되어버림

X_test[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']] = scaler.transform(X_test[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']])


X_test

In [ ]:
model = lgbm.LGBMClassifier()
model.fit(X_train,y_train)

In [ ]:
pred = model.predict(X_test)
pred

In [ ]:
submission = pd.read_csv('sample_submission.csv')

submission['quality'] = pred

submission

submission.to_csv('submission.csv',index=False)

# 보팅

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
enc.fit(train['type'])
train['type'] = enc.transform(train['type'])
test['type'] = enc.transform(test['type'])

In [ ]:
X_train = train.drop(['index','quality'], axis = 1)
X_test = test.drop(['index'], axis = 1)
y_train = train['quality']

In [ ]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
LGBM = LGBMClassifier()
XGB = XGBClassifier()
RF = RandomForestClassifier()

# VotingClassifier 정의
VC = VotingClassifier(estimators=[('rf',RF),('xgb',XGB),('lgbm',LGBM)],voting = 'soft')

In [ ]:
VC.fit(X_train,y_train)

In [ ]:
pred = VC.predict(X_test)

In [ ]:
pred

In [ ]:
submission = pd.read_csv('sample_submission.csv')

submission['quality'] = pred

submission

submission.to_csv('submission.csv',index=False)

## 정규화 후 보팅

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
enc.fit(train['type'])
train['type'] = enc.transform(train['type'])
test['type'] = enc.transform(test['type'])

In [ ]:
X_train = train.drop(['index','quality'], axis = 1)
X_test = test.drop(['index'], axis = 1)
y_train = train['quality']

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(X_train[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']])

X_train[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']] = scaler.transform(X_train[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']])


X_train

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(X_test[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']])  # 데이터프레임이 되어버림

X_test[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']] = scaler.transform(X_test[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']])


X_test

In [ ]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
LGBM = LGBMClassifier()
XGB = XGBClassifier()
RF = RandomForestClassifier()

# VotingClassifier 정의
VC = VotingClassifier(estimators=[('rf',RF),('xgb',XGB),('lgbm',LGBM)],voting = 'soft')

In [ ]:
VC.fit(X_train,y_train)

In [ ]:
pred = VC.predict(X_test)

In [ ]:
pred

In [ ]:
submission = pd.read_csv('sample_submission.csv')

submission['quality'] = pred

submission

submission.to_csv('submission.csv',index=False)

# GridSearch

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
RF = RandomForestClassifier()

In [ ]:
parameters = {'max_depth':[10], 'min_samples_split':[4]} # 결과를 알고 있음.

In [ ]:
grid_RF = GridSearchCV(RF, param_grid=parameters, cv=3, refit=True, return_train_score=True)
grid_RF.fit(X_train, y_train)

In [ ]:
print('GridSearchCV 최적 파라미터:', grid_RF.best_params_)

In [ ]:
estimator = grid_RF.best_estimator_
estimator

In [ ]:
pred = estimator.predict(X_test)
pred

In [ ]:
submission = pd.read_csv('sample_submission.csv')

submission['quality'] = pred

submission

submission.to_csv('submission.csv',index=False)

# 보팅 + 최적의 파라미터

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
enc.fit(train['type'])
train['type'] = enc.transform(train['type'])
test['type'] = enc.transform(test['type'])

In [ ]:
X_train = train.drop(['index','quality'], axis = 1)
X_test = test.drop(['index'], axis = 1)
y_train = train['quality']

In [ ]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
LGBM = LGBMClassifier()
XGB = XGBClassifier()
RF = RandomForestClassifier(max_depth=10, min_samples_split=4)

# VotingClassifier 정의
VC = VotingClassifier(estimators=[('rf',RF),('xgb',XGB),('lgbm',LGBM)],voting = 'soft')

In [ ]:
VC.fit(X_train,y_train)

In [ ]:
pred = VC.predict(X_test)

In [ ]:
pred

In [ ]:
submission = pd.read_csv('sample_submission.csv')

submission['quality'] = pred

submission

submission.to_csv('submission.csv',index=False)

# 보팅(모델 추가)

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
enc.fit(train['type'])
train['type'] = enc.transform(train['type'])
test['type'] = enc.transform(test['type'])

In [ ]:
X_train = train.drop(['index','quality'], axis = 1)
X_test = test.drop(['index'], axis = 1)
y_train = train['quality']

In [ ]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.neighbors import KNeighborsClassifier  

In [ ]:
LGBM = LGBMClassifier()
XGB = XGBClassifier()
RF = RandomForestClassifier()
lr_clf = LogisticRegression()
knn_clf = KNeighborsClassifier(n_neighbors=10)

# VotingClassifier 정의
VC = VotingClassifier(estimators=[('rf',RF),('xgb',XGB),('lgbm',LGBM),('LR', lr_clf),('KNN', knn_clf)],voting = 'soft')

In [ ]:
VC.fit(X_train,y_train)

In [ ]:
pred = VC.predict(X_test)

In [ ]:
pred

In [ ]:
submission = pd.read_csv('sample_submission.csv')

submission['quality'] = pred

submission

submission.to_csv('submission.csv',index=False)